# Zhong instances may need a quadratic number of SECs to be solved to optimality if $n = 1 \mod 3$

In this notebook, we validate the claims of the Section 5.4, claiming that the Rectilinear 3-Dimensional instances proposed by [1] may need a quadratic number of SECs to be solved to optimality if $n = 1 \mod 3$.

[1] Zhong, Xianghui. "Lower Bounds on the Integraliy Ratio of the Subtour LP for the Traveling Salesman Problem." arXiv preprint arXiv:2102.04765 (2021).

In [1]:
from ialg import *
import numpy as np
from scipy.spatial.distance import cdist

## Numerical tests validating our hypotheis

Let's start with some auxiliary functions

In [2]:
def generates_all_chains(Ls):
    m = len(Ls)
    chains = []
    for k in range(3, m + 1):
        for i in range(m - k + 1):
            chains.append(Ls[i:i + k])
    return chains

def generates_zhong_SECs(n):
    i = (n - 1) // 3 - 2
    G1 = list(range(i + 2))
    G2 = list(range(i + 2, 2 * i + 3))
    G3 = list(range(2 * i + 3, n))
    C1 = generates_all_chains(G1)
    C2 = generates_all_chains(G2)
    C3 = generates_all_chains(G3)
    C_all = C1 + C2 + C3
    return C_all

def derive_i_from_n(n):
    return (n - 1) // 3 - 2

def sum_first_k_integers(k):
    return k * (k + 1) // 2

def theoretical_number_of_SECs(n):
    i = derive_i_from_n(n)
    return sum_first_k_integers(i) + sum_first_k_integers(i - 1) + sum_first_k_integers(i + 2) # Note that is quadratic in i, hence, quadratic in n

def generate_rectilinear_3D_instance(n):
    i = derive_i_from_n(n)
    j = i - 1
    k = i + 2
    P = []
    assert i + j + k + 6 == n
    for s in range(0, i + 2):
        P.append([0, 0, s/(i + 1)])
    for s in range(0, j + 2):
        P.append([(1/(i+1)) + (1/(j+1)), 0, s/(j+1)])
    for s in range(0, k + 2):
        P.append([1 / (i + 1), 1 / (k + 1), s / (k + 1)])
    assert len(P) == n
    # Find the commont denominator
    delta = np.lcm.reduce([(i+1), (j+1), (k+1)])
    # Multiply everything by the common denominator
    P = np.array(P) * delta
    # Compute the distances
    C = cdist(P, P, 'minkowski', p=1)
    G = nx.Graph()
    for i in range(n):
        for j in range(i + 1, n):
            # Just check that all the valuse are integers
            G.add_edge(i, j, cost=int(C[i][j])) # We need to convert to integer
    return G
    

Now, let's test the constraints we have to asses if we have a tour - you may need a while for solving everything, remember that these are HARD instances

In [4]:
for n in [13, 16, 19, 22, 25, 28, 31, 34]:
    print(f"n = {n}")
    G = generate_rectilinear_3D_instance(n)
    # Compute the minimum number of SECs
    S_family, S_num, partitions, max_comp, runtime, bb_nodes = ialg(G, verbose=False)
    S_set_conjectured = theoretical_number_of_SECs(n)
    print("For n = ", n, " we need ", S_num, "SECs", "out of the ", S_set_conjectured, "conjectured")
    is_true = S_num == S_set_conjectured
    if not is_true:
        print("\tThe conjecture is wrong for n = ", n)
    else:
        print("\tRemoving one set at time...")
        # Check also that the set if minimal 
        found_error = False
        S_set_conjectured_list = generates_zhong_SECs(n)
        for s in range(S_num):
            # Consider a set of SECs without the s-th SEC
            S_set_test = [S_set_conjectured_list[i] for i in range(S_num) if i != s]
            # Solve TSP with just these SECs
            vals, components = mip(G, initial_subtours=S_set_test, return_components=True, verbose=False)
            if len(components) == 1:
                found_error = True
                print("\tThe conjectured set is not minimal")
                continue
        if not found_error:
            print("\tThe conjectured set is minimal")

n = 13
For n =  13  we need  14 SECs out of the  14 conjectured
	Removing one set at time...
	The conjectured set is minimal
n = 16
For n =  16  we need  15 SECs out of the  24 conjectured
	The conjecture is wrong for n =  16
n = 19
For n =  19  we need  37 SECs out of the  37 conjectured
	Removing one set at time...
	The conjectured set is minimal
n = 22
For n =  22  we need  53 SECs out of the  53 conjectured
	Removing one set at time...
	The conjectured set is minimal
n = 25
For n =  25  we need  72 SECs out of the  72 conjectured
	Removing one set at time...
	The conjectured set is minimal
n = 28
For n =  28  we need  94 SECs out of the  94 conjectured
	Removing one set at time...
	The conjectured set is minimal
n = 31
For n =  31  we need  119 SECs out of the  119 conjectured
	Removing one set at time...
	The conjectured set is minimal
n = 34
For n =  34  we need  147 SECs out of the  147 conjectured
	Removing one set at time...
	The conjectured set is minimal
